<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%AF%D0%BD%D0%B4%D0%B5%D0%BA%D1%81_%D0%9A%D0%B0%D1%80%D1%82%D0%B8%D0%BD%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/andron23/yandex-data.git

Cloning into 'yandex-data'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 17.10 MiB | 26.53 MiB/s, done.


In [ ]:
from zipfile import ZipFile

with ZipFile('yandex-data/data.zip', "r") as zip:
  zip.extractall(path='uncompressed')

In [ ]:
import pandas as pd

df = pd.read_json('uncompressed/data.json')

df.head()

,query,ts,platform
0,порно фото,1631786697,touch
1,малевич картины,1631806465,desktop
2,Секс,1631781583,touch
3,с днём рождения лена,1631771563,touch
4,зверополис порно,1631787599,touch


In [ ]:
df['query'] = df['query'].str.lower()

df['ts'] = pd.to_datetime(df['ts'], unit='s') + pd.Timedelta('03:00:00')

df.head()

,query,ts,platform,ts2
0,порно фото,2021-09-16 13:04:57,touch,2021-09-16
1,малевич картины,2021-09-16 18:34:25,desktop,2021-09-16
2,секс,2021-09-16 11:39:43,touch,2021-09-16
3,с днём рождения лена,2021-09-16 08:52:43,touch,2021-09-16
4,зверополис порно,2021-09-16 13:19:59,touch,2021-09-16


In [ ]:
import plotly.express as px

df['ts2'] = df['ts'].dt.strftime('%Y-%m-%d')

df1 = (
    df
    .groupby(['ts2'])
    ['query']
    .agg({'count'})
    .reset_index()
)

px.line(df1, x='ts2', y='count')

In [ ]:
min(df['ts']), max(df['ts'])

(Timestamp('2021-09-01 00:00:00'), Timestamp('2021-09-21 23:59:59'))

In [ ]:
(
    df
    .groupby(['platform', 'query'])
    .agg({'count'})
    .droplevel(1, axis='columns')
    .sort_values('ts', ascending=False)
    .groupby(['platform'])
    .head(10)
    .sort_values(['platform', 'ts'], ascending=False)
)

ts    ts2
platform query                                        
touch    порно                            10076  10076
         секс                              8262   8262
         с днём рождения женщине           4924   4924
         хентай                            4194   4194
         с днём рождения                   4123   4123
         с днём рождения мужчине           3669   3669
         с днем рождения                   3315   3315
         xxx                               3249   3249
         xnxx                              2767   2767
         доброе утро                       2342   2342
desktop  календарь 2021                    2810   2810
         таблица менделеева                2659   2659
         картинки                          1720   1720
         порно                             1408   1408
         английский алфавит                1352   1352
         обои на рабочий стол              1143   1143
         одноклассники (социальная сеть)   1116   1116
         секс                               947    947
         алфавит                            923    923
         таблица квадратов                  883    883

In [ ]:
(
    df
    .groupby(['wd'], as_index=False)
    ['ts']
    .agg({'count'})
)

,wd,count
0,1,180828
1,2,184308
2,3,186966
3,4,170886
4,5,163703
5,6,149694
6,7,167439


In [ ]:
df['wd'] = df['ts'].dt.dayofweek + 1

px.bar(
    (
        df
        .groupby(['wd'], as_index=False)
        ['ts']
        .agg({'count'})
    ),
    x='wd',
    y='count'
)

In [ ]:
df['hour'] = df['ts'].dt.hour

most_frequent_phrases = set()

for h in df['hour'].unique():
  most_frequent_phrases.update(
      df
      [df['hour'] == h]
      .groupby(['query'], as_index=False)
      ['ts']
      .agg({'count'})
      .sort_values('count', ascending=False)
      .head(3)
      ['query']
      .tolist()
  )

In [ ]:
most_frequent_phrases

{'xxx',
 'доброе утро',
 'календарь 2021',
 'порно',
 'с днём рождения',
 'с днём рождения женщине',
 'секс',
 'таблица менделеева',
 'хентай'}

In [ ]:
(
    df
    [df['query'].isin(most_frequent_phrases)]
    .groupby(['platform', 'query', 'hour'], as_index=False)
    .agg({'count'})
    .droplevel(1, axis='columns')
)

,platform,query,hour,ts,ts2,wd
0,desktop,xxx,0,8,8,8
1,desktop,xxx,1,5,5,5
2,desktop,xxx,2,4,4,4
3,desktop,xxx,3,1,1,1
4,desktop,xxx,5,2,2,2
...,...,...,...,...,...,...
423,touch,хентай,19,214,214,214
424,touch,хентай,20,264,264,264
425,touch,хентай,21,228,228,228
426,touch,хентай,22,280,280,280


In [ ]:
fig = px.line(
    df
    [df['query'].isin(most_frequent_phrases)]
    .groupby(['platform', 'query', 'hour'], as_index=False)
    .agg({'count'})
    .droplevel(1, axis='columns')
    ,
    x='hour',
    y='ts',
    color='query',
    facet_col='platform'
)

fig.update_yaxes(matches=None)

fig.show()

In [ ]:
df1 = (
    df
    .groupby(['platform', 'query'], as_index=False)
    ['ts']
    .agg({'count'})
    .pivot_table(
        index='query',
        columns='platform',
        values='count',
        fill_value=0
    )
    .sort_values('desktop', ascending=False)
)

df1['summa'] = df1['desktop'] + df1['touch']

df1 = df1[df1['summa'] > 15]

df1['desktop_rel'] = df1['desktop'] / max(df1['desktop'])
df1['touch_rel'] = df1['touch'] / max(df1['touch'])

df1['diff'] = df1['desktop']  - df1['touch']

df1

platform,desktop,touch,summa,desktop_rel,touch_rel,diff
query,,,,,,
календарь 2021,2810.0,2268.0,5078.0,1.000000,0.225089,542.0
таблица менделеева,2659.0,2278.0,4937.0,0.946263,0.226082,381.0
картинки,1720.0,1801.0,3521.0,0.612100,0.178742,-81.0
порно,1408.0,10076.0,11484.0,0.501068,1.000000,-8668.0
английский алфавит,1352.0,1749.0,3101.0,0.481139,0.173581,-397.0
...,...,...,...,...,...,...
опята ложные как отличить,0.0,24.0,24.0,0.000000,0.002382,-24.0
смешные картинки хорошего дня,0.0,37.0,37.0,0.000000,0.003672,-37.0
смешные картинки с добрым утром,0.0,21.0,21.0,0.000000,0.002084,-21.0


In [ ]:
px.box(df1, x='summa')

In [ ]:
px.box(df1, x='diff')

In [ ]:
import numpy as np

np.quantile(df1['diff'], 0.33), np.quantile(df1['diff'], 0.99)

(-16.0, 62.0)

In [ ]:
df1[(df1['diff'] > 35) | (df1['diff'] < -51)]

platform,desktop,touch,summa,desktop_rel,touch_rel,diff
query,,,,,,
календарь 2021,2810.0,2268.0,5078.0,1.000000,0.225089,542.0
таблица менделеева,2659.0,2278.0,4937.0,0.946263,0.226082,381.0
картинки,1720.0,1801.0,3521.0,0.612100,0.178742,-81.0
порно,1408.0,10076.0,11484.0,0.501068,1.000000,-8668.0
английский алфавит,1352.0,1749.0,3101.0,0.481139,0.173581,-397.0
...,...,...,...,...,...,...
открытки с днём знаний,0.0,57.0,57.0,0.000000,0.005657,-57.0
обои на телефон красивые,0.0,71.0,71.0,0.000000,0.007046,-71.0
онлайн казино,0.0,55.0,55.0,0.000000,0.005459,-55.0


In [ ]:
df2 = df1[(df1['diff'] > 35) | (df1['diff'] < -51)]

px.scatter(
    df2,
    x='desktop',
    y='touch'
)